# Chatbot Project Using Intents-Based Classification

## 1. Introduction
 In this notebook, we will build a simple intents-based chatbot that utilizes Natural Language Processing (NLP) techniques
 to understand user inputs and provide contextually relevant responses. We'll use machine learning models for intent classification
 and Streamlit for the user interface.
 Let's start by importing the required libraries.

In [ ]:

import pickle
import random
import ssl
import nltk
import re
import string
import json
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

 ## 2. Download Necessary NLTK Resources
 Before we begin, let's download the necessary NLTK resources for tokenization, stopword removal, and lemmatization.

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

## 3. Define Intents
 For this chatbot, we'll define a list of intents. Each intent will have:
 - A `tag` representing the intent category (e.g., "greeting")
 - `patterns` containing common user inputs related to that intent
 - `responses` with the chatbot's possible replies for that intent.

In [ ]:

intents = [
    {"tag": "greeting", "patterns": ["Hi", "Hello", "How are you?", "Hey"], "responses": ["Hello!", "Hi there!", "How can I help you?"]},
    {"tag": "goodbye", "patterns": ["Bye", "Goodbye", "See you later"], "responses": ["Goodbye!", "See you!", "Take care!"]},
    {"tag": "help", "patterns": ["I need help", "Can you assist me?", "I don't understand"], "responses": ["How can I assist you?", "I'm here to help!"]}
]

 ## 4. Preprocessing the Text
 We will define a preprocessing function to clean and tokenize user inputs.
 This will involve:
 - Converting text to lowercase
 - Tokenizing the text into words
 - Removing stopwords (common words like "the", "and") and punctuation
 - Lemmatizing the words (reducing words to their root form)

 Define stopwords and lemmatizer

In [ ]:

stopwords_set = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def regex_tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())  # Tokenize the text by words

def data_preprocess(text):
    tokens = regex_tokenize(text)  # Tokenize the text
    processed_tokens = [
        lemmatizer.lemmatize(word)  # Lemmatize each word
        for word in tokens
        if word not in string.punctuation and word not in stopwords_set  # Remove punctuation and stopwords
    ]
    return processed_tokens

 ## 5. Model Training
 Next, we will prepare the data for training a machine learning model. We will use the `TfidfVectorizer` to convert the user input patterns
 into numerical features, and then train a `MultinomialNB` classifier to predict the intent.

 Prepare training data

In [ ]:
patterns = []
labels = []
for intent in intents:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        labels.append(intent["tag"])

 Convert patterns into numerical data using TF-IDF vectorizer

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(patterns)

Train a Naive Bayes model

In [ ]:
model = MultinomialNB()
model.fit(X_train, labels)


## 6. Save Model and Vectorizer
 It's good practice to save the trained model and vectorizer so we can use them later without retraining. We will save them using `pickle`.

In [ ]:
with open('chatbot_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('chatbot_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

 ## 7. Chatbot Function
 Now we define the chatbot function that will take user input, preprocess it, and predict the intent using the trained model.

In [ ]:
def chatbot(user_inp):
    # Process the user input
    user_inp = data_preprocess(user_inp)  # Preprocess user input
    user_inp_vectorized = vectorizer.transform([' '.join(user_inp)])  # Vectorize the input
    
    # Predict the intent
    tag = model.predict(user_inp_vectorized)[0]
    
    # Find the corresponding response
    for intent in intents:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    
    return "Sorry, I didn't understand that."

 ## 8. Streamlit Application
 To make this chatbot accessible through a web interface, we will use Streamlit to create an interactive UI.
 Streamlit allows us to build applications with minimal effort.

In [ ]:
st.title("Intent-Based ChatBot")

if "messages" not in st.session_state:
    st.session_state.messages = []  # Initialize the session state to store chat messages

# Display previous messages in the chat
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Get user input
prompt = st.chat_input("Ask me here...")
if prompt:
    with st.chat_message("user"):
        st.markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})  # Add user message to session state

    # Get chatbot response
    ans = chatbot(prompt)
    with st.chat_message("assistant"):
        st.markdown(ans)
    st.session_state.messages.append({"role": "assistant", "content": ans})  # Add assistant response to session state

 ## 9. Future Improvements
 The current chatbot is quite basic. In the future, we can improve it by:
 - Expanding the dataset with more intents and patterns to make the chatbot more versatile.
 - Using advanced machine learning models, such as deep learning or BERT, to improve intent classification accuracy.
 - Adding the ability to handle multi-turn conversations for better user interactions.
 - Incorporating more complex natural language understanding features, such as named entity recognition (NER) and sentiment analysis.

 ## 10. Conclusion
 In this project, we successfully built an intents-based chatbot that uses NLP techniques to understand user input and generate contextually relevant responses.
 The chatbot is powered by a machine learning model (Naive Bayes) for intent classification and is deployed using Streamlit for an interactive user interface.
 With further improvements, this chatbot can be expanded to handle more complex interactions and be deployed in real-world applications.
